In [1]:
############################################################
# 
############################################################

# load some bits and pieces
import numpy as np
from numpy.linalg import solve
from numpy.linalg import lstsq
from numpy import log

import matplotlib
import matplotlib.pyplot as plt

import CoolProp as CP
from CoolProp.CoolProp import PropsSI

# Check: CoolProp version
print(CP.__version__)
print(CP.__gitrevision__)

# Constants
eps = 1e-3
kilo = 1e3
Mega = 1e6
golden = (1 + 5 ** 0.5) / 2
width = 12.5

6.1.1dev
6d076f9a634cd4b0991d075bb6aa0c0aa29ed8dd


In [2]:
# Calculation of the coefficients for the metastable region interpolation happens in this cell

# Set FluidName
FluidName = 'nButane'
AS = CP.AbstractState("HEOS", FluidName)
nPoints = 100

# Constants, triple and critical data
R0 = PropsSI('GAS_CONSTANT',FluidName)
MM = PropsSI('MOLAR_MASS',FluidName)
Rs = R0/MM
T_crt = PropsSI('T_CRITICAL',FluidName)
T_trp = PropsSI('T_TRIPLE',FluidName)
T_max = PropsSI('T_MAX',FluidName)

p_crt = PropsSI('P_CRITICAL',FluidName)
p_trp = PropsSI('P_TRIPLE',FluidName)
p_max = PropsSI('P_MAX',FluidName)

d_crt = PropsSI('RHOMASS_CRITICAL',FluidName)
d_trp_liq = PropsSI('D','T',T_trp,'Q',0,FluidName)
d_trp_vap = PropsSI('D','T',T_trp,'Q',1,FluidName)

v_crt = 1/d_crt
v_trp_liq = 1/d_trp_liq
v_trp_vap = 1/d_trp_vap

print("R0 = " + str(R0))
print("Rs = " + str(Rs))
print("MM = " + str(MM))
print("Rs = " + str(Rs))
print("T_crt = " + str(T_crt))
print("T_trp = " + str(T_trp))

# empty matrices 
iRows = nPoints**2
iCols = 11
myShape = (iRows,iCols)
A = np.empty(myShape)
B = np.empty(myShape)

# get a range for two input props
pRange = np.linspace(p_crt/100, p_max, num=nPoints)
TRange = np.linspace((T_crt+T_trp)/2, (T_crt+T_max)/2, num=nPoints)

idx = 0
# get values from CoolProp
for iP in range(0,nPoints):
    for iT in range(0,nPoints):        
        # update state
        AS.update(CP.PT_INPUTS, pRange[iP], TRange[iT]) 
        A[idx,0] = TRange[iT]
        A[idx,1] = pRange[iP]
        A[idx,2] = AS.rhomass()
        A[idx,3] = AS.smass()
        A[idx,4] = AS.umass()
        A[idx,5] = AS.hmass()
        idx = idx +1

A[:,6] = 1/A[:,2]     # v
A[:,7] = log(A[:,2])  # lnd
A[:,8] = log(A[:,6])  # lnv = -lnd
A[:,9] = log(A[:,1])  # lnp
A[:,10] = 1/(A[:,0])  # 1/T

# make a dimensionless copy
B[:,0] = A[:,0]/T_crt
B[:,1] = A[:,1]/p_crt
B[:,2] = A[:,2]/d_crt
B[:,3] = A[:,3]/Rs
B[:,4] = A[:,4]/Rs/T_crt
B[:,5] = A[:,5]/Rs/T_crt

B[:,6] = A[:,6]/v_crt
B[:,7] = log(B[:,2])  # lnd
B[:,8] = log(B[:,6])  # lnv = -lnd
B[:,9] = log(B[:,1])  # lnp
B[:,10] = 1/(B[:,0])  # Tc/T

np.savetxt("Surface_pT_A.csv", A, delimiter=",", header="T,p,d,s,u,h,v,lnd,lnv,lnp,Tinv", comments="")
np.savetxt("Surface_pT_B.csv", B, delimiter=",", header="T,p,d,s,u,h,v,lnd,lnv,lnp,Tinv", comments="")

R0 = 8.314472
Rs = 143.0515706563069
MM = 0.0581222
Rs = 143.0515706563069
T_crt = 425.125
T_trp = 134.895
